In [1]:

import os
import sys
import psutil
import pywinauto
print(sys.version)


3.12.8 (tags/v3.12.8:2dc476b, Dec  3 2024, 19:30:04) [MSC v.1942 64 bit (AMD64)]


In [48]:
# 获取微信pid
def get_process_pids(process_name='WeChat.exe'):
    PIDs= []
    for proc in psutil.process_iter():
        try:
            pinfo = proc.as_dict(attrs=['pid', 'name'])
        except psutil.NoSuchProcess:
            pass
        else:
            if process_name == pinfo['name']:
                PIDs.append( pinfo['pid'])
    return PIDs
# 获取指定进程窗口列表
def get_process_hwnd_list(process_name='WeChat.exe'):
    pids=get_process_pids(process_name)
    hwnd_list_res=[]
    for pid in pids:
        try:
            hwnd_list = pywinauto.findwindows.find_windows(process=pid)
        except Exception as e:
            print(e)
        else:
            hwnd_list_res+=hwnd_list
    return hwnd_list_res
# 窗口置于前
def bring_window_to_front_by_hwnd(hwnd):
    try:
        # 如果找到窗口句柄，选择第一个窗口
        # 使用 pywinauto 控制窗口
        app = pywinauto. Application().connect(handle=hwnd)
        window = app.window(handle=hwnd)
        # 将窗口置于前台
        window.set_focus()
        print(f"Window with PID {pid} has been brought to the front.")
    except Exception as e:
        print(f"Error: {e}")
    else:
        print(f"No windows found for PID {pid}.")
  


In [50]:
ex_window_pid=get_process_hwnd_list('WeChatAppEx.exe')[0]
bring_window_to_front_by_hwnd(ex_window_pid)

Error: name 'pid' is not defined


In [11]:
def connect_app(process_name):
    wechat_pid=get_process_pids(process_name)[0]
    app = pywinauto.Application(backend='uia').connect(process=wechat_pid)
    return app

In [ ]:

# wechat_app=connect_app('WeChat.exe')
get_process_pids('WeChat.exe')


33956

In [7]:
# get_process_pid('WeChatAppEx.exe')
app=connect_app('WeChatAppEx.exe')


In [23]:
bring_window_to_front_by_pid(get_process_pids('WeChatAppEx.exe')[8])

No windows found for PID 32576.


In [20]:
# 打开微信朋友圈
win = app['微信']
pyq_btn = win.child_window(title="朋友圈", control_type="Button")
cords = pyq_btn.rectangle()
pywinauto.mouse.click(button='left', coords=(cords.left + 10, cords.top + 10))

In [38]:

app.windows()

[<uiawrapper.UIAWrapper - '微信', Dialog, -2305705971332267035>]

In [25]:
moments_win=app['朋友圈']
print(moments_win.dump_tree())

Control Identifiers:

Dialog - '朋友圈'    (L720, T282, R1545, B1482)
['朋友圈', 'Dialog', '朋友圈Dialog', '朋友圈0', '朋友圈1']
child_window(title="朋友圈", control_type="Window")
   | 
   | Pane - ''    (L701, T263, R1564, B1501)
   | ['Pane', '2月2日在瑞中报告厅，我们不见不散[玫瑰]Pane', 'Pane0', 'Pane1', '2月2日在瑞中报告厅，我们不见不散[玫瑰]Pane0', '2月2日在瑞中报告厅，我们不见不散[玫瑰]Pane1']
   | 
   | Pane - ''    (L720, T282, R1545, B1482)
   | ['Pane2', '2月2日在瑞中报告厅，我们不见不散[玫瑰]Pane2']
   |    | 
   |    | Pane - ''    (L720, T282, R1545, B1482)
   |    | ['Pane3', '2月2日在瑞中报告厅，我们不见不散[玫瑰]Pane3']
   |    |    | 
   |    |    | ListBox - '朋友圈'    (L720, T282, R1545, B1482)
   |    |    | ['朋友圈ListBox', '朋友圈2', 'ListBox', 'ListBox0', 'ListBox1']
   |    |    | child_window(title="朋友圈", control_type="List")
   |    |    |    | 
   |    |    |    | Pane - ''    (L720, T228, R1545, B798)
   |    |    |    | ['Pane4']
   |    |    |    |    | 
   |    |    |    |    | Pane - ''    (L720, T228, R1545, B798)
   |    |    |    |    | ['Pane5']
   |    |  

In [27]:
import pyautogui
import time
time.sleep(5)
pyautogui.write('Hello, world!')

In [35]:
import win32gui
import win32ui
import win32con
import time
import numpy as np
from PIL import Image
import pyautogui

class WechatMomentsCapture:
    def __init__(self):
        self.hwnd = None
        self.scroll_step = 80  # 根据实际分辨率调整
        self.screenshot_list = []
        self.final_image = None

    def find_wechat_window(self):
        """查找微信窗口句柄"""
        self.hwnd = win32gui.FindWindow("Chrome_WidgetWin_0", "微信")
        if not self.hwnd:
            raise Exception("未找到微信窗口，请先打开微信")
        
        # 将窗口置前
        win32gui.SetForegroundWindow(self.hwnd)
        time.sleep(1)

    def capture_scroll(self):
        """滚动截图主逻辑"""
        # 定位朋友圈窗口位置
        rect = win32gui.GetWindowRect(self.hwnd)
        x, y, w, h = rect
        width = w - x
        height = h - y

        # 获取初始截图
        self._capture_area(x, y, width, height)
        
        # 自动滚动截图
        prev_page = None
        while True:
            # 执行滚动
            pyautogui.scroll(-self.scroll_step)
            time.sleep(0.5)
            
            # 获取当前页面截图
            current_page = self._capture_area(x, y, width, height)
            
            # 判断是否到达底部
            if self._is_same_image(prev_page, current_page):
                break
            
            self.screenshot_list.append(current_page)
            prev_page = current_page

    def _capture_area(self, x, y, width, height):
        """具体截图实现"""
        hwndDC = win32gui.GetWindowDC(self.hwnd)
        mfcDC = win32ui.CreateDCFromHandle(hwndDC)
        saveDC = mfcDC.CreateCompatibleDC()

        saveBitMap = win32ui.CreateBitmap()
        saveBitMap.CreateCompatibleBitmap(mfcDC, width, height)

        saveDC.SelectObject(saveBitMap)
        saveDC.BitBlt((0, 0), (width, height), mfcDC, (0, 0), win32con.SRCCOPY)

        bmpinfo = saveBitMap.GetInfo()
        bmpstr = saveBitMap.GetBitmapBits(True)

        im = Image.frombuffer(
            'RGB',
            (bmpinfo['bmWidth'], bmpinfo['bmHeight']),
            bmpstr, 'raw', 'BGRX', 0, 1
        )

        # 释放资源
        win32gui.DeleteObject(saveBitMap.GetHandle())
        saveDC.DeleteDC()
        mfcDC.DeleteDC()
        win32gui.ReleaseDC(self.hwnd, hwndDC)

        return im

    def _is_same_image(self, img1, img2, threshold=0.99):
        """判断图片相似度"""
        if img1 is None or img2 is None:
            return False
        
        # 转换为numpy数组进行比较
        arr1 = np.array(img1)
        arr2 = np.array(img2)
        
        # 计算相似度
        similarity = np.mean(arr1 == arr2)
        return similarity > threshold

    def merge_images(self):
        """合并所有截图"""
        total_height = sum(img.height for img in self.screenshot_list)
        max_width = max(img.width for img in self.screenshot_list)
        
        merged_image = Image.new('RGB', (max_width, total_height))
        y_offset = 0
        
        for img in self.screenshot_list:
            merged_image.paste(img, (0, y_offset))
            y_offset += img.height
        
        self.final_image = merged_image

    def save_image(self, filename):
        """保存最终图片"""
        if self.final_image:
            self.final_image.save(filename)
            print(f"截图已保存为：{filename}")

if __name__ == "__main__":
    try:
        capture_tool = WechatMomentsCapture()
        capture_tool.find_wechat_window()
        print("请手动切换到朋友圈界面...")
        time.sleep(5)  # 留出时间切换窗口
        
        capture_tool.capture_scroll()
        capture_tool.merge_images()
        capture_tool.save_image("wechat_moments_full.png")
        
    except Exception as e:
        print(f"发生错误：{str(e)}")

发生错误：(0, 'SetForegroundWindow', 'No error message is available')
